# * VINSIGHT : Metric Insight

## Parameter

In [114]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [149]:
''' Parameter Input '''

# v_metric_cd = 'B0R000100' #Total Revenue
# v_metric_cd = 'DB0R000100' #Total Revenue : DTAC
# v_metric_cd = 'TB0R000100' #Total Revenue : TRUE
# v_metric_cd = 'B0R000100GEO' #Total Revenue (Geo)
# v_metric_cd = 'DB0R000100GEO' #Total Revenue (Geo) : DTAC
# v_metric_cd = 'TB0R000100GEO' #Total Revenue (Geo) : TRUE

# v_metric_cd = 'B0R000101' #Mobile Revenue
# v_metric_cd = 'DB0R000101' #Mobile Revenue : DTAC
# v_metric_cd = 'TB0R000101' #Mobile Revenue : TMH
# v_metric_cd = 'B0R000101GEO' #Mobile Revenue (Geo)
# v_metric_cd = 'DB0R000101GEO' #Mobile Revenue (Geo) : DTAC
# v_metric_cd = 'TB0R000101GEO' #Mobile Revenue (Geo) : TMH

# v_metric_cd = 'B1R000100' #Prepaid Revenue
# v_metric_cd = 'DB1R000100' #Prepaid Revenue : DTAC
# v_metric_cd = 'TB1R000100' #Prepaid Revenue : TMH

# v_metric_cd = 'B2R000100' #Postpaid Revenue
# v_metric_cd = 'B2R010100' #Postpaid Revenue B2C
# v_metric_cd = 'B2R020100' #Postpaid Revenue B2B
# v_metric_cd = 'DB2R000100' #Postpaid Revenue : DTAC
# v_metric_cd = 'DB2R010100' #Postpaid Revenue B2C : DTAC
# v_metric_cd = 'DB2R020100' #Postpaid Revenue B2B : DTAC
# v_metric_cd = 'TB2R000100' #Postpaid Revenue : TMH
v_metric_cd = 'TB2R010100' #Postpaid Revenue B2C : TMH
# v_metric_cd = 'TB2R020100' #Postpaid Revenue B2B : TMH

# v_metric_cd = 'TB3R000100' #TOL Revenue

# v_metric_cd = 'TB4R000100' #TVS Revenue

# v_metric_cd = 'B0R00010001CS' #Total Inflow M1
# v_metric_cd = 'DB0R00010001CS' #Total Inflow M1 : DTAC
# v_metric_cd = 'TB0R00010001CS' #Total Inflow M1 : TRUE

# v_metric_cd = 'B1R000900CS' #Prepaid Inflow M1
# v_metric_cd = 'DB1R000900CS' #Prepaid Inflow M1 : DTAC
# v_metric_cd = 'TB1R000900CS' #Prepaid Inflow M1 : TMH

# v_metric_cd = 'B2R010500CS' #Postpaid Inflow M1 B2C
# v_metric_cd = 'B2R020500CS' #Postpaid Inflow M1 B2B
# v_metric_cd = 'DB2R010500CS' #Postpaid Inflow M1 B2C : DTAC
# v_metric_cd = 'DB2R020500CS' #Postpaid Inflow M1 B2B : DTAC
# v_metric_cd = 'TB2R010500CS' #Postpaid Inflow M1 B2C : TMH
# v_metric_cd = 'TB2R020500CS' #Postpaid Inflow M1 B2B : TMH

# v_metric_cd = 'TB3R000601CS' #TOL Inflow M1 Connected : Consumer
# v_metric_cd = 'TB3R000601D1CS' #TOL Inflow M1 Connected : Consumer (Install Location)
# v_metric_cd = 'TB3R000602CS' #TOL Inflow M1 Connected : DataService
# v_metric_cd = 'TB3R000602D1CS' #TOL Inflow M1 Connected : DataService (Install Location)

# v_metric_cd = 'TB4R001004CS' #TVS CMDU Inflow M1
# v_metric_cd = 'TB4R001700CS' #TVS Now Inflow M1

# v_period_grp = 'Yearly' #Yearly, Quarterly, Monthly, Weekly, Daily

# v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_period_grp: {v_period_grp}')
# print(f'   -> v_start_date: {v_start_date}')


Parameter input...

   -> v_metric_cd: TB2R010100


## Metric Transaction

In [116]:
''' Execute transaction '''

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, AREA_TYPE, AREA_CD, AREA_NAME
        , ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_WK, ACTUAL_AGG_MTH, ACTUAL_AGG_QTR, ACTUAL_AGG_YR 
        , TARGET_SNAP, TARGET_AGG_WK, TARGET_AGG_MTH, TARGET_AGG_QTR, TARGET_AGG_YR 
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = '{v_metric_cd}'
    --AND TM_KEY_DAY >= v_start_date
    --ORDER BY TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Data as of 2025-05-29, 15:55:13



   -> DataFrame : 110320 rows, 24 columns


In [410]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = str(chk_src_df['TM_KEY_YR'].drop_duplicates().shift().max().astype(int))

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = str(chk_src_df['TM_KEY_QTR'].drop_duplicates().shift().max().astype(int))

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().shift().max().astype(int))

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = str(chk_src_df['TM_KEY_WK'].drop_duplicates().shift().max().astype(int))

curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
prev_day = str(chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int))

In [411]:
# ''' Sample Data '''

# chk_src_df.tail()

### Overview

In [412]:
''' Yearly Summary '''

yearly_df = chk_src_df.copy()
yearly_df['P_TARGET'] = np.where(yearly_df['AREA_CD']=='P', yearly_df['TARGET_SNAP'], 0)
yearly_df['C'] = np.where(yearly_df['AREA_CD']=='C', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['P'] = np.where(yearly_df['AREA_CD']=='P', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['G'] = np.where(yearly_df['AREA_TYPE']=='G', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['H'] = np.where(yearly_df['AREA_TYPE']=='H', yearly_df['ACTUAL_SNAP'], 0)
yearly_df['HH'] = np.where(yearly_df['AREA_TYPE']=='HH', yearly_df['ACTUAL_SNAP'], 0)

yearly_df = yearly_df.groupby(['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
yearly_df = yearly_df.fillna(0).sort_values(by=['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
yearly_df['ACTUAL_AS_OF'] = yearly_df['ACTUAL_AS_OF'].astype(int)

yearly_df_display = yearly_df.copy()
mod_col_list = yearly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    yearly_df_display[col] = yearly_df_display[col].apply(lambda x: format(x, ',.0f'))

yearly_df_display

,TM_KEY_YR,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH
0,2023,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20231231,"33,245,012,566",0,"32,854,818,250","32,543,728,912","32,737,963,624",0
1,2024,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20241231,"39,985,016,061","39,407,414,546","39,407,414,546","39,340,777,769","39,340,655,852","39,340,655,852"
2,2025,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,"16,418,129,375","16,348,284,331","16,348,284,331","16,341,460,324","16,341,446,181","16,341,446,181"


In [413]:
''' Quarterly Summary '''

quarterly_df = chk_src_df.copy()
quarterly_df['P_TARGET'] = np.where(quarterly_df['AREA_CD']=='P', quarterly_df['TARGET_SNAP'], 0)
quarterly_df['C'] = np.where(quarterly_df['AREA_CD']=='C', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['P'] = np.where(quarterly_df['AREA_CD']=='P', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['G'] = np.where(quarterly_df['AREA_TYPE']=='G', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['H'] = np.where(quarterly_df['AREA_TYPE']=='H', quarterly_df['ACTUAL_SNAP'], 0)
quarterly_df['HH'] = np.where(quarterly_df['AREA_TYPE']=='HH', quarterly_df['ACTUAL_SNAP'], 0)

quarterly_df = quarterly_df.groupby(['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
quarterly_df = quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
quarterly_df['ACTUAL_AS_OF'] = quarterly_df['ACTUAL_AS_OF'].astype(int)

quarterly_df_display = quarterly_df.copy()
mod_col_list = quarterly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    quarterly_df_display[col] = quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))

quarterly_df_display

,TM_KEY_QTR,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH
0,20231,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20230331,"3,316,616,982",0,"3,233,961,809","3,207,262,907","3,226,149,679",0
1,20232,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20230630,"9,990,498,738",0,"9,786,703,099","9,704,815,161","9,762,505,383",0
2,20233,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20230930,"9,954,064,200",0,"9,885,358,090","9,798,875,131","9,857,447,809",0
3,20234,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2024-04-23 12:22:16,20231231,"9,983,832,647",0,"9,948,795,252","9,832,775,713","9,891,860,754",0
4,20241,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20240331,"10,038,537,351","9,850,451,722","9,850,451,722","9,833,627,564","9,833,558,358","9,833,558,358"
5,20242,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20240630,"9,936,699,487","9,838,450,875","9,838,450,875","9,821,126,021","9,821,074,464","9,821,074,464"
6,20243,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20240930,"9,987,507,408","9,834,028,205","9,834,028,205","9,818,595,523","9,818,594,946","9,818,594,946"
7,20244,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20241231,"10,022,271,815","9,884,483,743","9,884,483,743","9,867,428,661","9,867,428,084","9,867,428,084"
8,20251,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250331,"10,046,030,199","9,989,547,936","9,989,547,936","9,982,743,196","9,982,733,231","9,982,733,231"
9,20252,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,"6,372,099,176","6,358,736,395","6,358,736,395","6,358,717,128","6,358,712,950","6,358,712,950"


In [414]:
''' Monthly Summary '''

v_tm_key_yr = curr_yr
# v_tm_key_yr = 2025

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_YR']==v_tm_key_yr].copy()
monthly_df['P_TARGET'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['TARGET_SNAP'], 0)
monthly_df['C'] = np.where(monthly_df['AREA_CD']=='C', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['P'] = np.where(monthly_df['AREA_CD']=='P', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['G'] = np.where(monthly_df['AREA_TYPE']=='G', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['H'] = np.where(monthly_df['AREA_TYPE']=='H', monthly_df['ACTUAL_SNAP'], 0)
monthly_df['HH'] = np.where(monthly_df['AREA_TYPE']=='HH', monthly_df['ACTUAL_SNAP'], 0)

monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['ACTUAL_AS_OF'] = monthly_df['ACTUAL_AS_OF'].astype(int)

monthly_df_display = monthly_df.copy()
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

monthly_df_display

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH
0,202501,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250131,"3,331,779,353","3,336,770,490","3,336,770,490","3,329,988,784","3,329,984,467","3,329,984,467"
1,202502,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250228,"3,347,051,673","3,333,288,223","3,333,288,223","3,333,276,337","3,333,273,586","3,333,273,586"
2,202503,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250331,"3,367,199,173","3,319,489,222","3,319,489,222","3,319,478,075","3,319,475,179","3,319,475,179"
3,202504,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250430,"3,358,275,973","3,371,023,854","3,371,023,854","3,371,013,229","3,371,011,525","3,371,011,525"
4,202505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,"3,013,823,203","2,987,712,541","2,987,712,541","2,987,703,898","2,987,701,426","2,987,701,426"


In [415]:
''' Weekly Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==v_tm_key_mth].copy()
weekly_df['P_TARGET'] = np.where(weekly_df['AREA_CD']=='P', weekly_df['TARGET_SNAP'], 0)
weekly_df['C'] = np.where(weekly_df['AREA_CD']=='C', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['P'] = np.where(weekly_df['AREA_CD']=='P', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['G'] = np.where(weekly_df['AREA_TYPE']=='G', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['H'] = np.where(weekly_df['AREA_TYPE']=='H', weekly_df['ACTUAL_SNAP'], 0)
weekly_df['HH'] = np.where(weekly_df['AREA_TYPE']=='HH', weekly_df['ACTUAL_SNAP'], 0)

weekly_df = weekly_df.groupby(['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
weekly_df = weekly_df.fillna(0).sort_values(by=['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
weekly_df['ACTUAL_AS_OF'] = weekly_df['ACTUAL_AS_OF'].astype(int)

weekly_df_display = weekly_df.copy()
mod_col_list = weekly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    weekly_df_display[col] = weekly_df_display[col].apply(lambda x: format(x, ',.0f'))

weekly_df_display

,TM_KEY_WK,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,P_TARGET,C,P,G,H,HH
0,2025018,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250502,"486,249,148","479,993,614","479,993,614","479,992,695","479,992,048","479,992,048"
1,2025019,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250510,"723,600,099","732,158,937","732,158,937","732,155,565","732,155,565","732,155,565"
2,2025020,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250516,"756,360,805","747,032,582","747,032,582","747,031,543","747,029,817","747,029,817"
3,2025021,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250525,"1,047,613,151","1,028,527,408","1,028,527,408","1,028,524,095","1,028,523,995","1,028,523,995"
4,2025022,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,0,0,0,0,0,0


In [416]:
''' Daily Summary '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = 202501

daily_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==v_tm_key_mth].copy()
daily_df['P_TARGET'] = np.where(daily_df['AREA_CD']=='P', daily_df['TARGET_SNAP'], 0)
daily_df['C'] = np.where(daily_df['AREA_CD']=='C', daily_df['ACTUAL_SNAP'], 0)
daily_df['P'] = np.where(daily_df['AREA_CD']=='P', daily_df['ACTUAL_SNAP'], 0)
daily_df['G'] = np.where(daily_df['AREA_TYPE']=='G', daily_df['ACTUAL_SNAP'], 0)
daily_df['H'] = np.where(daily_df['AREA_TYPE']=='H', daily_df['ACTUAL_SNAP'], 0)
daily_df['HH'] = np.where(daily_df['AREA_TYPE']=='HH', daily_df['ACTUAL_SNAP'], 0)

daily_df = daily_df.groupby(['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'PPN_TM':'max', 'P_TARGET':'sum', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
daily_df = daily_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

daily_df_display = daily_df.copy()
mod_col_list = daily_df_display.iloc[:, 5:].columns.tolist()
for col in mod_col_list:
    daily_df_display[col] = daily_df_display[col].apply(lambda x: format(x, ',.0f'))

daily_df_display

,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P_TARGET,C,P,G,H,HH
0,20250501,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
1,20250502,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,"486,249,148","479,993,614","479,993,614","479,992,695","479,992,048","479,992,048"
2,20250503,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
3,20250504,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
4,20250505,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
5,20250506,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,"267,643,342","287,826,439","287,826,439","287,825,682","287,825,682","287,825,682"
6,20250507,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
7,20250508,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
8,20250509,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,0,0,0,0,0,0
9,20250510,Postpaid,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,"455,956,757","444,332,498","444,332,498","444,329,884","444,329,884","444,329,884"


### Area Summary

In [417]:
''' Period Input '''

v_period_grp = 'Monthly' #Yearly, Quarterly, Monthly, Weekly, Daily

print(f'\nPeriod input...\n')
print(f'   -> v_period_grp: {v_period_grp}')


Period input...

   -> v_period_grp: Monthly


In [418]:
def agg_period(v_period_grp):

    # Get : Parameter
    period_grp = v_period_grp
    result = 0

    if period_grp == 'Yearly': result = curr_yr
    elif period_grp == 'Quarterly': result = curr_qtr
    elif period_grp == 'Monthly': result = curr_mth
    elif period_grp == 'Weekly': result = curr_wk
    else: result = curr_day
 
    return result

# Check Result
agg_period(v_period_grp)

202505

In [463]:
''' Auto Prep Aggregate by Period '''

if v_period_grp == 'Yearly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Quarterly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_QTR', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Monthly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_MTH', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Weekly':
    raw_period_df = chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_WK', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
elif v_period_grp == 'Daily':
    raw_period_df = chk_src_df.groupby(['TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME']).agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'ACTUAL_SNAP':'sum', 'ACTUAL_AGG_MTH':'sum', 'ACTUAL_AGG_QTR':'sum', 'ACTUAL_AGG_YR':'sum', 'TARGET_SNAP':'sum', 'TARGET_AGG_MTH':'sum', 'TARGET_AGG_QTR':'sum', 'TARGET_AGG_YR':'sum'}).copy().reset_index()
    raw_period_df = raw_period_df[['TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'ACTUAL_SNAP', 'TARGET_SNAP']]
else:
    print(f'\n*** Aggregate Period Input ? ***')


''' Add Columns '''

raw_period_df['GAP'] = raw_period_df['ACTUAL_SNAP'] - raw_period_df['TARGET_SNAP']
raw_period_df['%ACH'] = raw_period_df['ACTUAL_SNAP'] / raw_period_df['TARGET_SNAP'] * 100
# raw_period_df['%PORTION'] = raw_period_df['ACTUAL_SNAP'] / raw_period_df.groupby([raw_period_df.columns[0], 'AREA_TYPE'])['ACTUAL_SNAP'].transform('sum') * 100
# raw_period_df

raw_period_df_display = raw_period_df.copy()
raw_period_df_display = raw_period_df_display.fillna(0).sort_values(by=[raw_period_df_display.columns[0], 'METRIC_CD', 'AREA_TYPE', 'AREA_CD']).reset_index(drop=True)
raw_period_df_display['ACTUAL_AS_OF'] = raw_period_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = raw_period_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    raw_period_df_display[col] = raw_period_df_display[col].apply(lambda x: format(x, ',.0f'))
raw_period_df_display['%ACH'] = raw_period_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
# raw_period_df_display['%PORTION'] = raw_period_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
# raw_period_df_display

In [464]:
''' P : Nationwide Summary '''

p_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
p_summary_df = p_summary_df.loc[p_summary_df['AREA_TYPE']=='P']
p_summary_df = p_summary_df.fillna(0).sort_values(by=[p_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
p_summary_df['%PORTION'] = p_summary_df['ACTUAL_SNAP'] / p_summary_df.groupby('AREA_TYPE')['ACTUAL_SNAP'].transform('sum') * 100
# p_summary_df = p_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

p_summary_df_display = p_summary_df.copy()
p_summary_df_display['ACTUAL_AS_OF'] = p_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = p_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    p_summary_df_display[col] = p_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
p_summary_df_display['%ACH'] = p_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
p_summary_df_display['%PORTION'] = p_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
p_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,P,P,Nationwide,"2,987,712,541","3,013,823,203","-26,110,662",99.13,100.00


In [465]:
''' G : 8 Region Summary '''

g_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
g_summary_df = g_summary_df.loc[g_summary_df['AREA_TYPE']=='G']
g_summary_df['%PORTION'] = g_summary_df['ACTUAL_SNAP'] / g_summary_df.groupby('AREA_TYPE')['ACTUAL_SNAP'].transform('sum') * 100
# g_summary_df = g_summary_df.fillna(0).sort_values(by=[g_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
g_summary_df = g_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

g_summary_df_display = g_summary_df.copy()
g_summary_df_display['ACTUAL_AS_OF'] = g_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = g_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    g_summary_df_display[col] = g_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
g_summary_df_display['%ACH'] = g_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
g_summary_df_display['%PORTION'] = g_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
g_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX3,East,"533,771,528","442,351,575","91,419,953",120.67,17.87
1,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX1,BMA : West,"470,171,676","425,263,907","44,907,769",110.56,15.74
2,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX2,BMA : East,"549,316,230","513,979,580","35,336,650",106.88,18.39
3,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX7,"Central, West","368,864,182","375,804,769","-6,940,587",98.15,12.35
4,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX8,South,"246,828,442","252,153,393","-5,324,951",97.89,8.26
5,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX4,North,"319,339,878","366,849,169","-47,509,291",87.05,10.69
6,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX5,Northeast 1,"278,338,516","332,057,945","-53,719,429",83.82,9.32
7,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-28 23:53:59,20250525,G,GX6,Northeast 2,"221,073,447","305,362,865","-84,289,418",72.40,7.40


In [466]:
''' H : 64 Province Summary '''

h_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
h_summary_df = h_summary_df.loc[h_summary_df['AREA_TYPE']=='H']
h_summary_df['%PORTION'] = h_summary_df['ACTUAL_SNAP'] / h_summary_df.groupby('AREA_TYPE')['ACTUAL_SNAP'].transform('sum') * 100
# h_summary_df = h_summary_df.fillna(0).sort_values(by=[h_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
h_summary_df = h_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

h_summary_df_display = h_summary_df.copy()
h_summary_df_display['ACTUAL_AS_OF'] = h_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = h_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    h_summary_df_display[col] = h_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
h_summary_df_display['%ACH'] = h_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
h_summary_df_display['%PORTION'] = h_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
h_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,003,"SMP : Bang Bo, Bang Sao Thong, Bang Phli","78,433,760","62,458,118","15,975,642",125.58,2.63
1,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,21Z,Rayong,"66,899,516","54,686,569","12,212,948",122.33,2.24
2,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,20B,Chon Buri 2,"108,531,060","89,766,846","18,764,214",120.90,3.63
3,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,007,"PTT : Mueang Pathum Thani, Lat Lum Kaeo, Sam K...","81,537,617","68,840,871","12,696,746",118.44,2.73
4,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,83Z,Phuket,"46,057,374","39,044,010","7,013,364",117.96,1.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,44X,"Maha Sarakham, Roi Et","50,709,755","76,247,219","-25,537,464",66.51,1.70
60,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,48X,"Sakon Nakhon, Nakhon Phanom","33,829,226","51,610,270","-17,781,044",65.55,1.13
61,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,31Z,Buri Ram,"34,512,872","53,073,581","-18,560,709",65.03,1.16
62,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 01:02:55,20250528,H,37X,"Yasothon, Amnat Charoen","19,316,996","30,261,860","-10,944,864",63.83,0.65


In [467]:
''' HH : 96 Cluster Summary '''

hh_summary_df = raw_period_df[raw_period_df[raw_period_df.columns[0]]==agg_period(v_period_grp)]
hh_summary_df = hh_summary_df.loc[hh_summary_df['AREA_TYPE']=='HH']
hh_summary_df['%PORTION'] = hh_summary_df['ACTUAL_SNAP'] / hh_summary_df.groupby('AREA_TYPE')['ACTUAL_SNAP'].transform('sum') * 100
# hh_summary_df = hh_summary_df.fillna(0).sort_values(by=[hh_summary_df.columns[0], 'METRIC_CD', 'AREA_CD']).reset_index(drop=True)
hh_summary_df = hh_summary_df.fillna(0).sort_values(by='%ACH', ascending=False).reset_index(drop=True)

hh_summary_df_display = hh_summary_df.copy()
hh_summary_df_display['ACTUAL_AS_OF'] = hh_summary_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = hh_summary_df_display.iloc[:, 8:11].columns.tolist()
for col in mod_col_list:
    hh_summary_df_display[col] = hh_summary_df_display[col].apply(lambda x: format(x, ',.0f'))
hh_summary_df_display['%ACH'] = hh_summary_df_display['%ACH'].apply(lambda x: format(x, ',.2f'))
hh_summary_df_display['%PORTION'] = hh_summary_df_display['%PORTION'].apply(lambda x: format(x, ',.2f'))
hh_summary_df_display

,TM_KEY_MTH,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,AREA_TYPE,AREA_CD,AREA_NAME,ACTUAL_SNAP,TARGET_SNAP,GAP,%ACH,%PORTION
0,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,903060,"SMP : Bang Bo, Bang Sao Thong, Bang Phli","78,433,760","62,458,118","15,975,642",125.58,2.63
1,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,903055,Rayong,"66,899,516","54,686,569","12,212,948",122.33,2.24
2,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,903052,Chon Buri 2,"108,531,060","89,766,846","18,764,214",120.90,3.63
3,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,908011,"PTT : Mueang Pathum Thani, Lat Lum Kaeo, Sam K...","81,537,617","68,840,871","12,696,746",118.44,2.73
4,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,906106,Phuket,"46,057,374","39,044,010","7,013,364",117.96,1.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,910091,Kalasin,"20,185,934","31,611,000","-11,425,066",63.86,0.68
92,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,909085,Nong Bua Lam Phu,"7,705,456","12,270,408","-4,564,951",62.80,0.26
93,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,910089,Amnat Charoen,"7,745,236","12,364,967","-4,619,732",62.64,0.26
94,202505,TB2R010100,Postpaid Revenue B2C : TMH,2025-05-29 03:00:52,20250528,HH,910096,Surin,"24,146,298","40,988,793","-16,842,495",58.91,0.81


### Area Insight

In [293]:
# ...